In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2024-04-25 06:00:00')


C:\Users\Victus\AppData\Local\Temp\ipykernel_14096\2041109657.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)


In [5]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/655531
Connected. Call `.close()` to terminate connection gracefully.


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/655531/featurestores/651354/featuregroups/time_series_hourly_feature_group). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":270009,"usrMsg":"feature group name: time_series_hourly_feature_group feature group version: 3","errorMsg":"Featuregroup wasn\'t found."}', error code: 270009, error msg: Featuregroup wasn't found., user msg: feature group name: time_series_hourly_feature_group feature group version: 3

In [ ]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

In [ ]:
predictions['pickup_hour'] = current_date
predictions

### Save these predictions in the feature store, so they can be later consumed by our Streamlit app

In [ ]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [ ]:
feature_group.insert(predictions, write_options={"wait_for_job": False})